<a href="https://colab.research.google.com/github/mohdrahil7/image-Segmentation/blob/master/unet%2Baugmen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#training the unet architecture along with data augmentation
#training and testing on color images of real size
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
#implementation of fcn-8 code
from keras.layers import *
from keras import Model
from matplotlib import pyplot as plt
import cv2
import os
import numpy as np

Using TensorFlow backend.


In [0]:
os.chdir('/content/drive')

In [0]:
os.chdir('My Drive')

In [0]:
os.chdir('dataset')

In [0]:
#reading the training data 
#reading x and y data
x=np.load('Skin_TestX.npy')
y=np.load('Skin_TestY.npy')
#reading the test_data
x_test=np.load('Skin_ValidX.npy')
y_test=np.load('Skin_ValidY.npy')

In [0]:
from sklearn.model_selection import train_test_split
x,validx,y,validy=train_test_split(x,y,test_size=0.2,random_state=28,shuffle=True) 

In [22]:
print(x.shape)
print(y.shape)
print(validx.shape)
print(validy.shape)
print(x_test.shape)
print(y_test.shape)

(480, 192, 256, 3)
(480, 192, 256, 1)
(120, 192, 256, 3)
(120, 192, 256, 1)
(150, 192, 256, 3)
(150, 192, 256, 1)


In [23]:
#reshaping
y=y.reshape(-1,192,256,1)
validy=validy.reshape(-1,192,256,1)
y_test=y_test.reshape(-1,192,256,1)
print(y.shape)
print(validy.shape)
print(y_test.shape)

(480, 192, 256, 1)
(120, 192, 256, 1)
(150, 192, 256, 1)


In [0]:
#performing the normalization of taking each value between 0 and 1
y=y.astype('float32')/255
y_test=y_test.astype('float32')/255
validy=validy.astype('float32')/255

In [0]:
#normalizing the training data
x_mean=np.mean(x)
x_std=np.std(x)
x=x-x_mean
x=x/x_std

In [0]:
#normalizing validation data
validx=validx-x_mean
validx=validx/x_std

In [0]:
#normalizing test data
x_test=x_test-x_mean
x_test=x_test/x_std

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(featurewise_center=False,samplewise_center=False,rotation_range=30,featurewise_std_normalization=False,
                           samplewise_std_normalization=False,horizontal_flip=True,zoom_range=[0.8,0.9])
#creating augmented image for both x image and y image
img_datagen=ImageDataGenerator(datagen)
img_mask=ImageDataGenerator(datagen)
bs=32 #creating batch size of 32
seed=215

In [0]:
img_gen=img_datagen.flow(x,batch_size=bs,seed=seed,shuffle=True)
mask_gen=img_mask.flow(y,batch_size=bs,seed=seed,shuffle=True)
#putting both x and y augmented images into one zip flip
img_zip=zip(img_gen,mask_gen)

In [41]:
#creating a Unet Network but since we are not creating the sequentail network so we need model API to create model

def create_model():
  input_img=Input(shape=(192,256,3))
  
  #block_1
  z=Conv2D(32,(3,3),padding='same',input_shape=(128,128,1),activation='relu',name='block1_conv1')(input_img)
  conv1=Conv2D(32,(3,3),padding='same',activation='relu',name='block1_conv2')(z)
  z=MaxPool2D(pool_size=(2,2),strides=(2,2),name='block1_maxpooling')(conv1)

  #block_2
  z=Conv2D(64,(3,3),padding='same',activation='relu',name='block2_conv1')(z)
  conv2=Conv2D(64,(3,3),padding='same',activation='relu',name='block2_conv2')(z)
  z=MaxPool2D(pool_size=(2,2),strides=(2,2),name='block2_maxpooling')(conv2)

  #block_3
  z=Conv2D(128,(3,3),padding='same',activation='relu',name='block3_conv1')(z)
  z=Conv2D(128,(3,3),padding='same',activation='relu',name='block3_conv2')(z)
  up1=Conv2DTranspose(128,(2,2),strides=(2,2),activation='relu',padding='same',name='upsampling')(z)

  concat1=concatenate([up1,conv2])
  #block_4
  z=Conv2D(64,(3,3),padding='same',activation='relu',name='block4_conv1')(concat1)
  z=Conv2D(64,(3,3),padding='same',activation='relu',name='block4_conv2')(z)
  up2=Conv2DTranspose(64,(2,2),strides=(2,2),activation='relu',padding='same',name='upsampling2')(z)
  
  concat2=concatenate([up2,conv1])
  #block_5
  z=Conv2D(32,(3,3),padding='same',activation='relu',name='block5_conv1')(concat2)
  z=Conv2D(32,(3,3),padding='same',activation='relu',name='block5_conv2')(z)
  z=Dropout(0.5)(z)
  final=Conv2D(1,(1,1),activation='relu',padding='same' ,name='final')(z)

  model1=Model(input_img,final)
  return model1

model=create_model()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 192, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 192, 256, 32) 896         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 192, 256, 32) 9248        block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_maxpooling (MaxPooling2D (None, 96, 128, 32)  0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [0]:
os.chdir('/content/drive/My Drive')

In [0]:
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [0]:
#creating callbacks
#creating the callbacks and checkpoints to save best model
from keras.callbacks import EarlyStopping,ModelCheckpoint
es=EarlyStopping(monitor="val_loss",mode='min',patience=10 ,verbose=1)
cp=ModelCheckpoint('Unet_color_aug.h5',monitor="val_loss",mode='min',verbose=1,save_best_only='true')

In [47]:
history=model.fit_generator(datagen.flow(x,y,batch_size=32,seed=bs),epochs=100,callbacks=([es,cp]),validation_data=(validx,validy),steps_per_epoch=(len(x)//bs))

Epoch 1/100
15/15 [==============================] - 7s 439ms/step - loss: 3.5117e-06 - accuracy: 0.7717 - val_loss: 4.1031e-06 - val_accuracy: 0.7332

Epoch 00001: val_loss did not improve from 0.00000
Epoch 2/100
15/15 [==============================] - 6s 422ms/step - loss: 3.5118e-06 - accuracy: 0.7717 - val_loss: 4.1031e-06 - val_accuracy: 0.7332

Epoch 00002: val_loss did not improve from 0.00000
Epoch 3/100
15/15 [==============================] - 6s 412ms/step - loss: 3.5118e-06 - accuracy: 0.7717 - val_loss: 4.1031e-06 - val_accuracy: 0.7332

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/100
15/15 [==============================] - 6s 419ms/step - loss: 3.5117e-06 - accuracy: 0.7717 - val_loss: 4.1031e-06 - val_accuracy: 0.7332

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/100
15/15 [==============================] - 6s 416ms/step - loss: 3.5117e-06 - accuracy: 0.7717 - val_loss: 4.1031e-06 - val_accuracy: 0.7332

Epoch 00005: val_loss did not improv